In [1]:
import selenium
from selenium import webdriver
from selenium.webdriver.support.select import Select
import time
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd

In [2]:
change_date_selector_id = "choose"
datatable_selector_class = "kose-yazarlari-endeksi-table"
page_url = "https://mediacat.com/mediacat-kose-yazarlari-endeksi/"

In [3]:
options = webdriver.ChromeOptions()
# options.add_argument("--headless")
browser = webdriver.Chrome("chromedriver.exe", options=options)
page = browser.get(page_url)

In [24]:
def get_dates():
    dates = []
    dropdown = browser.find_element_by_id(change_date_selector_id)
    options = dropdown.find_elements_by_tag_name("option")
    dates = [option.get_attribute("value") for option in options]
    print(dates)
    return dates
dates = get_dates()

['2020-04-27', '2020-04-26', '2020-04-25', '2020-04-24', '2020-04-23', '2020-04-22', '2020-04-21', '2020-04-20', '2020-04-19', '2020-04-18', '2020-04-17', '2020-04-16', '2020-04-15', '2020-04-14', '2020-04-13', '2020-04-12', '2020-04-11', '2020-04-10', '2020-04-09', '2020-04-08', '2020-04-07', '2020-04-06', '2020-04-05', '2020-04-04', '2020-04-03', '2020-04-02', '2020-04-01', '2020-03-31', '2020-03-30', '2020-03-29', '2020-03-28', '2020-03-27', '2020-03-26', '2020-03-25', '2020-03-24', '2020-03-23', '2020-03-22', '2020-03-21', '2020-03-20', '2020-03-19', '2020-03-18', '2020-03-17', '2020-03-16', '2020-03-15', '2020-03-14', '2020-03-13', '2020-03-12', '2020-03-11', '2020-03-10', '2020-03-09', '2020-03-08', '2020-03-07', '2020-03-06', '2020-03-05', '2020-03-04', '2020-03-03', '2020-03-02', '2020-03-01', '2020-02-29', '2020-02-28', '2020-02-27', '2020-02-26', '2020-02-25', '2020-02-24', '2020-02-23', '2020-02-22', '2020-02-21', '2020-02-20', '2020-02-19', '2020-02-18', '2020-02-17', '2020

In [26]:
def change_date(date):
    dropdown = Select(browser.find_element_by_id(change_date_selector_id))
    dropdown.select_by_value(date)
change_date(dates[2])

In [91]:
def get_data_table():
    data_table = browser.find_element_by_class_name(datatable_selector_class)
    rows = data_table.find_elements_by_class_name("table-item")
    day = []
    for row in rows:
        data = {}
        data["rank"] = row.find_element_by_class_name("count").text
        content = row.find_element_by_class_name("content")
        data["columnist"] = content.find_elements_by_tag_name("b")[0].text
        data["newspaper"] = content.find_elements_by_tag_name("span")[0].text.replace("/ ", "")
        data["article_title"] = content.text.split("\n")[1]
        data["article_url"] = row.find_elements_by_tag_name("a")[0].get_attribute("href")
        data["facebook"] = row.find_elements_by_tag_name("td")[1].get_attribute("textContent").strip()
        data["twitter"] = row.find_elements_by_tag_name("td")[2].get_attribute("textContent").strip()
        data["total"] = row.find_elements_by_tag_name("td")[3].get_attribute("textContent").strip()
        day.append(data)
    return day
get_data_table()

[{'rank': '1',
  'columnist': 'Yılmaz Özdil',
  'newspaper': 'Sözcü',
  'article_title': 'Bilim Kurulu’na yalvarıyorum ...',
  'article_url': 'https://www.sozcu.com.tr/2020/yazarlar/yilmaz-ozdil/bilim-kuruluna-yalvariyorum-5762180/',
  'facebook': '7532',
  'twitter': '3231',
  'total': '10763'},
 {'rank': '2',
  'columnist': 'Soner Yalçın',
  'newspaper': 'Sözcü',
  'article_title': 'Bir görüşleri daha çöktü ...',
  'article_url': 'https://www.sozcu.com.tr/2020/yazarlar/soner-yalcin/bir-gorusleri-daha-coktu-5762053/',
  'facebook': '800',
  'twitter': '2923',
  'total': '3723'},
 {'rank': '3',
  'columnist': 'Arslan BULUT',
  'newspaper': 'Yeni Çağ',
  'article_title': 'Koronavirüsten daha tehlikeli yaklaşımlar! ...',
  'article_url': 'https://www.yenicaggazetesi.com.tr/koronavirusten-daha-tehlikeli-yaklasimlar-55529yy.htm',
  'facebook': '592',
  'twitter': '3017',
  'total': '3609'},
 {'rank': '4',
  'columnist': 'Fuat Uğur',
  'newspaper': 'Türkiye',
  'article_title': 'Türkiye bu 

In [27]:
def get_data_table_pd():
    data_table = browser.find_element_by_class_name(datatable_selector_class)
    data_table_pd = pd.read_html(data_table.get_attribute("outerHTML").replace("<br>", "é"))
    return data_table_pd[0]
data_table_pd = get_data_table_pd()
# data_table_pd.to_csv("deneme.csv")

In [22]:
data_table_pd[0][1]

'1 Arslan BULUT /  Yeni Çağ é  Bu işte küresel bir sahtekârlık var! ...'

In [31]:
days = []
for date in dates:
    print(date)
    wait = WebDriverWait(browser, 10)
    change_date(date)
    # wait.until(EC.visibility_of_element_located((By.CLASS_NAME, datatable_selector_class)))
    # This line waits for ajax process to be completed. Awesome
    
    wait.until(lambda browser: browser.execute_script('return jQuery.active') == 0)
    print("reloaded")
    day = get_data_table_pd()
    day["date"] = date
    # day = get_data_table()
    days.append(day)

2020-04-27
reloaded
2020-04-26
reloaded
2020-04-25
reloaded
2020-04-24
reloaded
2020-04-23
reloaded
2020-04-22
reloaded
2020-04-21
reloaded
2020-04-20
reloaded
2020-04-19
reloaded
2020-04-18
reloaded
2020-04-17
reloaded
2020-04-16
reloaded
2020-04-15
reloaded
2020-04-14
reloaded
2020-04-13
reloaded
2020-04-12
reloaded
2020-04-11
reloaded
2020-04-10
reloaded
2020-04-09
reloaded
2020-04-08
reloaded
2020-04-07
reloaded
2020-04-06
reloaded
2020-04-05
reloaded
2020-04-04
reloaded
2020-04-03
reloaded
2020-04-02
reloaded
2020-04-01
reloaded
2020-03-31
reloaded
2020-03-30
reloaded
2020-03-29
reloaded
2020-03-28
reloaded
2020-03-27
reloaded
2020-03-26
reloaded
2020-03-25
reloaded
2020-03-24
reloaded
2020-03-23
reloaded
2020-03-22
reloaded
2020-03-21
reloaded
2020-03-20
reloaded
2020-03-19
reloaded
2020-03-18
reloaded
2020-03-17
reloaded
2020-03-16
reloaded
2020-03-15
reloaded
2020-03-14
reloaded
2020-03-13
reloaded
2020-03-12
reloaded
2020-03-11
reloaded
2020-03-10
reloaded
2020-03-09
reloaded


In [67]:
days_pd = pd.concat(days)

In [39]:
print(len(dates))
len(days_pd.date.unique())

2367


2367

In [72]:
# Rename cols
days_pd.columns = ["content", "facebook", "twitter", "total", "date"]
# Drop rows with index of 0. Those were html table header
days_pd.drop(0, axis=0, inplace=True)
# There was a row with null values
null_rows = days_pd[days_pd["content"].apply(lambda x: isinstance(x, float))].index
days_pd.drop(null_rows, inplace=True)


AttributeError: 'float' object has no attribute 'split'

In [121]:
# Rank is the first item in strip list which strip based on empty space
days_pd["rank"] = days_pd["content"].apply(lambda x: x.split(" ")[0]).head(20)
# Divide by "/", divide by " ", join all except the first one. First one is rank, the rest supposed to be columnist
days_pd["columnist"] = days_pd["content"].apply(lambda x: " ".join(x.split("/")[0].split(" ")[1:]))
# Divide by "é", take the first half, divide by "/", take the second half (to be error proof, took the last part)
days_pd["newspaper"] = days_pd["content"].apply(lambda x: x.split("é")[0].split("/")[-1])
# Split by "é", take the right side of it
days_pd["column_title"] = days_pd["content"].apply(lambda x: x.split("é")[1])


In [123]:
days_pd.to_excel("export.xlsx")